In [90]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import quad
import unittest
import scipy.special as sp
import ipywidgets as widgets
from IPython.display import display, clear_output

In [91]:
class Laguerre:
    def __init__(self, beta=2, sigma=4, n=20, eps=0.001, t_max=100, t_points=1000):
        self._beta = beta
        self._sigma = sigma
        self._n = n
        self._eps = eps
        self._t_max = t_max
        self._t_points = t_points



    @property
    def beta(self):
        return self._beta
    
    @beta.setter
    def beta(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Beta must be a number")
        if value <= 0:
            raise ValueError("Beta must be a positive number")
        self._beta = value

    @property
    def sigma(self):
        return self._sigma
    
    @sigma.setter
    def sigma(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Sigma must be a number")
        elif value <= 0:
            raise ValueError("Sigma must be a positive number")
        else:
            self._sigma = value

    @property
    def n(self):
        return self._n
    
    @n.setter
    def n(self, value):
        if not isinstance(value, int):
            raise ValueError("n must be an integer")
        if value < 0:
            raise ValueError("n must be a non-negative integer")
        else:
            self._n = value

    @property
    def eps(self):
        return self._eps
    
    @eps.setter
    def eps(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Epsilon must be a number")
        if value <= 0:
            raise ValueError("Epsilon must be a positive number")
        else:
            self._eps = value

    @property
    def t_max(self):
        return self._t_max
    
    @t_max.setter
    def t_max(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("t_max must be a number")
        elif value <= 0:
            raise ValueError("t_max must be a positive number")
        else:
            self._t_max = value

    @property
    def t_points(self):
        return self._t_points
    
    @t_points.setter
    def t_points(self, value):
        if not isinstance(value, int):
            raise ValueError("t_points must be an integer")
        if value <= 0:
            raise ValueError("t_points must be a positive integer")
        else:
            self._t_points = value

    def laguerre(self, t, n):
        l0 = np.sqrt(self.sigma) * (np.exp(-self.beta * t / 2))
        l1 = np.sqrt(self.sigma) * (1 - self.sigma * t) * (np.exp(-self.beta * t / 2))
        if n == 0:
            return l0
        elif n == 1:
            return l1
        else:
            lnext = (2 * 2 - 1 - t * self.sigma) / 2 * l1 - (2 - 1) / 2 * l0
            for j in range(3, n + 1):
                l0 = l1
                l1 = lnext
                lnext = (2 * j - 1 - t * self.sigma) / j * l1 - (j - 1) / j * l0 
            return lnext

    def experiment(self):
        suitable_t = None
        T = np.linspace(0, self.t_max, self.t_points)
        N = range(0, self.n + 1)
        for t in T:
            is_t_suitable = all(abs(self.laguerre(t, n)) < self.eps for n in N)
            if is_t_suitable and suitable_t is None:
                suitable_t = t
                break
        return suitable_t, N

    def transform_laguerre(self, f, n):
        def func(t):
            alpha = self.sigma-self.beta 
            
            return f(t)*self.laguerre(t,n) * np.exp(-alpha*t)
        t = self.experiment()[0]

        return quad(func,0,t)[0]


    def tabulate_transformation(self, f, N):
        result = []
        for n in range(N+1):
            result.append(self.transform_laguerre(f, n))
        return result


    def tabulate_laguerre(self, T, num_points=100):
        t_values = np.linspace(0, T, num_points)
        laguerre_values = [self.laguerre(t, self.n) for t in t_values]
        return t_values, laguerre_values
    

In [92]:
class LaguerreWidgets:
    def __init__(self, beta=2, sigma=4, n=20, eps=0.001, t_max=100, t_points=1000):
        self._beta = beta
        self._sigma = sigma
        self._n = n
        self._eps = eps
        self._t_max = t_max
        self._t_points = t_points

    @property
    def beta(self):
        return self._beta
    
    @beta.setter
    def beta(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Beta must be a number")
        if value <= 0:
            raise ValueError("Beta must be a positive number")
        self._beta = value

    @property
    def sigma(self):
        return self._sigma
    
    @sigma.setter
    def sigma(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Sigma must be a number")
        elif value <= 0:
            raise ValueError("Sigma must be a positive number")
        else:
            self._sigma = value

    @property
    def n(self):
        return self._n
    
    @n.setter
    def n(self, value):
        if not isinstance(value, int):
            raise ValueError("n must be an integer")
        if value < 0:
            raise ValueError("n must be a non-negative integer")
        else:
            self._n = value

    @property
    def eps(self):
        return self._eps
    
    @eps.setter
    def eps(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("Epsilon must be a number")
        if value <= 0:
            raise ValueError("Epsilon must be a positive number")
        else:
            self._eps = value

    @property
    def t_max(self):
        return self._t_max
    
    @t_max.setter
    def t_max(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("t_max must be a number")
        elif value <= 0:
            raise ValueError("t_max must be a positive number")
        else:
            self._t_max = value

    @property
    def t_points(self):
        return self._t_points
    
    @t_points.setter
    def t_points(self, value):
        if not isinstance(value, int):
            raise ValueError("t_points must be an integer")
        if value <= 0:
            raise ValueError("t_points must be a positive integer")
        else:
            self._t_points = value

    def laguerre_widget(self):

        n_slider = widgets.IntSlider(min=0, max=10, continuous_update=False, description='n')
        t_slider = widgets.FloatSlider(min=0, max=1, step=0.01, continuous_update=False, description='t')


        output = widgets.Output()
        def update_result(change):
            with output:
                laguerre = Laguerre()
                result = laguerre.laguerre(t_slider.value, n_slider.value)
                print(f"Результат для n={n_slider.value}, t={t_slider.value}: {result}")


        n_slider.observe(update_result, 'value')
        t_slider.observe(update_result, 'value')

        display(widgets.HBox([n_slider, t_slider]))
        display(output)

        l0 = np.sqrt(self.sigma) * (np.exp(-self.beta * t_slider.value / 2))
        l1 = np.sqrt(self.sigma) * (1 - self.sigma * t_slider.value) * (np.exp(-self.beta * t_slider.value / 2))
        if n_slider.value == 0:
            return l0
        elif n_slider.value == 1:
            return l1
        else:
            lnext = (2 * 2 - 1 - t_slider.value * self.sigma) / 2 * l1 - (2 - 1) / 2 * l0
            for j in range(3, n_slider.value + 1):
                l0 = l1
                l1 = lnext
                lnext = (2 * j - 1 - t_slider.value * self.sigma) / j * l1 - (j - 1) / j * l0 
            return lnext
        

    def experiment_widget(self):
        n_slider_exp = widgets.IntSlider(min=1, max=50, continuous_update=False, description='n')
        beta_slider_exp = widgets.FloatSlider(min=0, max=10, step=0.1, value=2, continuous_update=False, description='beta')
        sigma_slider_exp = widgets.FloatSlider(min=0, max=10, step=0.1, value=4, continuous_update=False, description='sigma')
        eps_value = 0.001
        eps_slider_exp = widgets.FloatSlider(min=0.001, max=0.001, step=0.001, value=eps_value, continuous_update=False, description='eps', disabled=True)
        t_max_slider_exp = widgets.IntSlider(min=10, max=200, step=10, value=100, continuous_update=False, description='t_max')
        t_points_slider_exp = widgets.IntSlider(min=100, max=2000, step=100, value=1000, continuous_update=False, description='t_points')

        output_exp = widgets.Output()

        def update_experiment(change):
            with output_exp:
                laguerre = Laguerre()
                clear_output(wait=True)
                laguerre.n = n_slider_exp.value
                laguerre.beta = beta_slider_exp.value
                laguerre.sigma = sigma_slider_exp.value
                laguerre.eps = eps_value
                laguerre.t_max = t_max_slider_exp.value
                laguerre.t_points = t_points_slider_exp.value

                t, N = laguerre.experiment()
                df = pd.DataFrame(laguerre.laguerre(t=t,n=n) for n in N)
                display(df)

        n_slider_exp.observe(update_experiment, 'value')
        beta_slider_exp.observe(update_experiment, 'value')
        sigma_slider_exp.observe(update_experiment, 'value')
        t_max_slider_exp.observe(update_experiment, 'value')
        t_points_slider_exp.observe(update_experiment, 'value')

        display(widgets.VBox([n_slider_exp, beta_slider_exp, sigma_slider_exp, eps_slider_exp, t_max_slider_exp, t_points_slider_exp]))
        display(output_exp)

    def transform_laguerre(self, f, n):
        def func(t):
            alpha = self.sigma-self.beta 
            
            return f(t)*self.laguerre(t,n) * np.exp(-alpha*t)
        t = self.experiment()[0]

        return quad(func,0,t)[0]


    def tabulate_transformation(self, f, N):
        result = []
        for n in range(N+1):
            result.append(self.transform_laguerre(f, n))
        return result


    def tabulate_laguerre(self, T, num_points=100):
        t_values = np.linspace(0, T, num_points)
        laguerre_values = [self.laguerre(t, self.n) for t in t_values]
        return t_values, laguerre_values
    
    def tabulate_transformation_widget(self):
        n_slider = widgets.IntSlider(min=0, max=10, continuous_update=False, description='n')
        beta_slider = widgets.FloatSlider(min=0, max=10, step=0.1, value=2, continuous_update=False, description='beta')
        sigma_slider = widgets.FloatSlider(min=0, max=10, step=0.1, value=4, continuous_update=False, description='sigma')
        T_slider = widgets.FloatSlider(min=1, max=10, step=0.1, value=5, continuous_update=False, description='T')
        n_point_slider = widgets.IntSlider(min=10, max=1000, step=10, value=100, continuous_update=False, description='num_points')

        output_df = widgets.Output()

        def update_df(change):
            with output_df:
                clear_output(wait=True)
                laguerre = Laguerre()
                laguerre.beta = beta_slider.value
                laguerre.sigma = sigma_slider.value

                t_values, laguerre_values = laguerre.tabulate_laguerre(T_slider.value,n_point_slider.value)
                df = pd.DataFrame(laguerre_values, index=t_values, columns=['Laguerre'])
                display(df)

        n_slider.observe(update_df, 'value')
        beta_slider.observe(update_df, 'value')
        sigma_slider.observe(update_df, 'value')
        T_slider.observe(update_df, 'value')
        n_point_slider.observe(update_df, 'value')

        display(widgets.VBox([n_slider, beta_slider, sigma_slider, T_slider, n_point_slider]))
        display(output_df)

In [93]:
class LaguerreInverse(Laguerre):
    def __init__(self, beta=2, sigma=4, n=20, eps=0.001, t_max=100, t_points=1000):
        super().__init__(beta, sigma, n, eps, t_max, t_points)

    def inverse_laguerre(self, h, t):
        result = 0
        for i, coeff in enumerate(h):
            result += coeff * super().laguerre(t, i)
        return result

    def tabulate_inverse(self, func,transformed_values,t1=0,t2=2*np.pi):
        T = np.linspace(t1,t2,1000)
        result =[]
        correct_values = []
        for t in T:
            result.append(self.inverse_laguerre(transformed_values,t))
            correct_values.append(func(t))
        return result,correct_values
    
    def inverse_laguerre_widget(self, f, transformed_values):
        def update_inverse_laguerre_plot(n):
            transformed = self.tabulate_transformation(f, n)
            reversed_transform_values, correct = self.tabulate_inverse(f, transformed)

            fig, ax = plt.subplots(1, 2, figsize=(20, 5))

            ax[0].plot(transformed, label="Пряме перетворення Лагерра(коефіцієнти)")
            ax[0].legend()

            ax[1].plot(np.linspace(0, 2*np.pi, 1000), reversed_transform_values, label='Обернене перетворення Лагерра')
            ax[1].plot(np.linspace(0, 2*np.pi, 1000), correct, label='Застосована функція', linestyle="--")
            ax[1].legend()

            plt.show()


        n_slider_inverse = widgets.IntSlider(min=1, max=20, continuous_update=False, description='n')

        output_inverse = widgets.Output()

        def update_inverse(change):
            with output_inverse:
                clear_output(wait=True)
                n_value = n_slider_inverse.value
                update_inverse_laguerre_plot(n_value)

        n_slider_inverse.observe(update_inverse, 'value')

        display(widgets.VBox([n_slider_inverse]))
        display(output_inverse)
    

In [94]:
class LaguerrePlotsWidgets(Laguerre):
    def __init__(self, beta=2, sigma=4, n=20, eps=0.001, t_max=100, t_points=1000):
        super().__init__(beta, sigma, n, eps, t_max, t_points)


    def make_plots(self, T, N):
        fig, ax = plt.subplots(figsize=(10, 10))

        for n in range(N + 1):
            t_values = np.linspace(0, T, self.t_points)
            laguerre_values = [super().laguerre(t, n) for t in t_values]
            ax.plot(t_values, laguerre_values, label=f"n={n}")

        ax.set_xlabel("t")
        ax.set_ylabel("l(t)")
        ax.set_title("Laguerre Functions")
        plt.grid()
        ax.legend()
        plt.show()

    def make_plots_widget(self):
        T_slider_plots = widgets.FloatSlider(min=1, max=10, step=0.1, value=4, continuous_update=False, description='T')
        N_slider_plots = widgets.IntSlider(min=1, max=20, continuous_update=False, description='N')

        output_plots = widgets.Output()

        def update_plots(change):
            with output_plots:

                clear_output(wait=True)
                T_value = T_slider_plots.value
                N_value = N_slider_plots.value -1
                self.make_plots(T_value, N_value)

        T_slider_plots.observe(update_plots, 'value')
        N_slider_plots.observe(update_plots, 'value')

        display(widgets.VBox([T_slider_plots, N_slider_plots]))
        display(output_plots)
            

In [95]:
temp = LaguerreWidgets()

temp.laguerre_widget()


Output()

2.0

In [96]:
temp1 = LaguerreWidgets()
temp1.experiment_widget()

Output()

In [97]:
temp2 = LaguerreWidgets()

temp2.tabulate_transformation_widget()


Output()

In [98]:
plots_widget = LaguerrePlotsWidgets()
plots_widget.make_plots_widget()

Output()

In [100]:
def f(t):
    if t >= 0 and t <= 2*np.pi:
        return np.sin(t-np.pi/2) + 1
    else: 
        return 0

t = LaguerreInverse()
t.inverse_laguerre_widget(f, [1 for i in range(21)])

Output()